In [2]:
import pandas as pd
import pyodbc

In [3]:
def create_connection():
    connection = None
    try:
        connection = pyodbc.connect(
            'Driver={ODBC Driver 17 for SQL Server};'
            'Server=username;'  # username
            'Trusted_Connection=yes;')
    except Exception as e:
        print('Database Connectivity failed.', e)
    return connection

In [4]:
conn = create_connection()


# I was having issue that is why i have used this technique

# Task 1

<b>Display the names of the unique launch sites in the space mission</b>

In [9]:
q = "select distinct Launch_Site from spacex"
pd.read_sql(q, conn)

,Launch_Site
0,CCAFS LC-40
1,CCAFS SLC-40
2,KSC LC-39A
3,VAFB SLC-4E


# Task 2

<b>Display 5 records where launch sites begin with the string 'CCA'</b>

In [68]:
# I was getting error while using LIMIT that is why i used TOP 5

q = " SELECT TOP 5 * FROM SPACEX WHERE Launch_Site LIKE 'CCA%'"


pd.read_sql(q, conn)

,Date,Time_UTC,Booster_Version,Launch_Site,Payload,PAYLOAD_MASS_KG,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


# Task 3

<b>Display the total payload mass carried by boosters launched by NASA (CRS)</b>

In [28]:

q = '''SELECT SUM(PAYLOAD_MASS_KG) AS total_payload_mass_kg
FROM spacex
WHERE CUSTOMER = 'NASA (CRS)'
'''
pd.read_sql(q, conn)

,total_payload_mass_kg
0,45596


# Task 4

<b>Display average payload mass carried by booster version F9 v1.1</b>

In [39]:
q = '''
SELECT AVG(PAYLOAD_MASS_KG) AS avg_payload_mass_kg
FROM spacex
WHERE Booster_Version = 'F9 v1.1'
'''
pd.read_sql(q, conn)

,avg_payload_mass_kg
0,2928


# Task 5

<b>List the date when the first successful landing outcome in ground pad was acheived.</b>

In [40]:
q = '''SELECT MIN(DATE) AS first_successful_landing_date
FROM SPACEX
WHERE Landing_Outcome = 'Success (ground pad)'
'''
pd.read_sql(q, conn)

,first_successful_landing_date
0,2015-12-22


# Task 6

<b>List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000</b>

In [42]:
q = '''SELECT Booster_Version
FROM SPACEX
WHERE Landing_Outcome = 'Success (drone ship)'
    AND (PAYLOAD_MASS_KG BETWEEN 4000 AND 6000)
'''
pd.read_sql(q, conn)

,Booster_Version
0,F9 FT B1022
1,F9 FT B1026
2,F9 FT B1021.2
3,F9 FT B1031.2


# Task 7

<b>List the total number of successful and failure mission outcomes</b>

In [44]:
q  = '''SELECT MISSION_OUTCOME, COUNT(*) AS total_number
FROM SPACEX
GROUP BY MISSION_OUTCOME
'''
pd.read_sql(q, conn)

,MISSION_OUTCOME,total_number
0,Failure (in flight),1
1,Success,99
2,Success (payload status unclear),1


# Task 8

<b>List the names of the booster_versions which have carried the maximum payload mass. Use a subquery</b>

In [47]:
q = '''
SELECT DISTINCT Booster_Version, PAYLOAD_MASS_KG
FROM SPACEX
WHERE PAYLOAD_MASS_KG = (
    SELECT MAX(PAYLOAD_MASS_KG)
    FROM SPACEX)
'''
pd.read_sql(q, conn)

,Booster_Version,PAYLOAD_MASS_KG
0,F9 B5 B1048.4,15600
1,F9 B5 B1048.5,15600
2,F9 B5 B1049.4,15600
3,F9 B5 B1049.5,15600
4,F9 B5 B1049.7,15600
5,F9 B5 B1051.3,15600
6,F9 B5 B1051.4,15600
7,F9 B5 B1051.6,15600
8,F9 B5 B1056.4,15600
9,F9 B5 B1058.3,15600


# Task 9

<b>List the failed landing_outcomes in drone ship, their booster versions, and launch site names for in year 2015</b>

In [48]:
q = '''
SELECT LANDING_OUTCOME, BOOSTER_VERSION, LAUNCH_SITE
FROM SPACEX
WHERE LANDING_OUTCOME = 'Failure (drone ship)' AND YEAR(DATE) = '2015'
'''
pd.read_sql(q, conn)

,LANDING_OUTCOME,BOOSTER_VERSION,LAUNCH_SITE
0,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
1,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


# Task 10

<b>Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order</b>

In [49]:
q = '''
SELECT LANDING_OUTCOME, COUNT(LANDING_OUTCOME) AS total_number
FROM SPACEX
WHERE DATE BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY LANDING_OUTCOME
ORDER BY total_number DESC
'''
pd.read_sql(q, conn)

,LANDING_OUTCOME,total_number
0,No attempt,10
1,Failure (drone ship),5
2,Success (drone ship),5
3,Success (ground pad),3
4,Controlled (ocean),3
5,Uncontrolled (ocean),2
6,Failure (parachute),2
7,Precluded (drone ship),1
